In [ ]:
import mediapipe as mp 
import cv2
import glob
import csv
import os
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import argparse
#from numpy.linalg import norm

In [ ]:
mp_drawing = mp.solutions.drawing_utils   
mp_drawing_styles = mp.solutions.drawing_styles  
mp_hands = mp.solutions.hands 

In [ ]:
#reading images
output_directory = r'C:\Uni\HiWi\PlayGround\ImageVersion2\OutputImages'
#output_directory = r'/datasets_nas/elma1294/results'
if not os.path.exists(output_directory):
    os.makedirs(output_directory)
    
#file = r'/datasets_nas/elma1294/example_tape/e4sm_tape-2021-07-20_15-31-35.523041_png.shuttleFront/*.jpg'
file = r'C:\Uni\HiWi\PlayGround\ImageVersion2\TestImages\*.jpg'

kinect_2D = r'C:\Uni\HiWi\PlayGround\ImageVersion2\results3Dto2Dnew.csv'
# 3d to 2d 
#kinect_output_file = baiad javabe 3D to 2d ro bezani inja
compare_mediapipe_kinect = os.path.join(output_directory, 'compare.csv')
mediapipe_csv = os.path.join(output_directory, 'mediapipe.csv')
filled_mediapipe_csv = os.path.join(output_directory, mediapipe_csv.replace('.csv','_fill.csv'))
merged_mediapipe_kinect = os.path.join(output_directory, 'merged_mediapipe_kinect.csv')

images = glob.glob(file)
output_images = []

In [ ]:
def make_results_dict(timestamp, which_hand, hand):
    mDict = {}
    mDict['timestamp'] = timestamp
    #type of mp_hands.HandLandmark is enum
    for handLandmark in mp_hands.HandLandmark:
        name = str(handLandmark.name).replace('HandLandmark.', '')
        mDict['{}_{}.x'.format(name, which_hand)] = hand.landmark[handLandmark.value].x
        mDict['{}_{}.y'.format(name, which_hand)] = hand.landmark[handLandmark.value].y
        mDict['{}_{}.z'.format(name, which_hand)] = hand.landmark[handLandmark.value].z
    return mDict

In [ ]:
header =['timestamp',
         'WRIST_LEFT.x','WRIST_LEFT.y','WRIST_LEFT.z',
         'THUMB_CMC_LEFT.x','THUMB_CMC_LEFT.y','THUMB_CMC_LEFT.z',
         'THUMB_MCP_LEFT.x','THUMB_MCP_LEFT.y','THUMB_MCP_LEFT.z',
         'THUMB_IP_LEFT.x','THUMB_IP_LEFT.y','THUMB_IP_LEFT.z',
         'THUMB_TIP_LEFT.x','THUMB_TIP_LEFT.y','THUMB_TIP_LEFT.z',
         'INDEX_FINGER_MCP_LEFT.x','INDEX_FINGER_MCP_LEFT.y','INDEX_FINGER_MCP_LEFT.z',
         'INDEX_FINGER_PIP_LEFT.x','INDEX_FINGER_PIP_LEFT.y','INDEX_FINGER_PIP_LEFT.z',
         'INDEX_FINGER_DIP_LEFT.x','INDEX_FINGER_DIP_LEFT.y','INDEX_FINGER_DIP_LEFT.z',
         'INDEX_FINGER_TIP_LEFT.x','INDEX_FINGER_TIP_LEFT.y','INDEX_FINGER_TIP_LEFT.z',
         'MIDDLE_FINGER_MCP_LEFT.x','MIDDLE_FINGER_MCP_LEFT.y','MIDDLE_FINGER_MCP_LEFT.z',
         'MIDDLE_FINGER_PIP_LEFT.x','MIDDLE_FINGER_PIP_LEFT.y','MIDDLE_FINGER_PIP_LEFT.z',
         'MIDDLE_FINGER_DIP_LEFT.x','MIDDLE_FINGER_DIP_LEFT.y','MIDDLE_FINGER_DIP_LEFT.z',
         'MIDDLE_FINGER_TIP_LEFT.x','MIDDLE_FINGER_TIP_LEFT.y','MIDDLE_FINGER_TIP_LEFT.z',
         'RING_FINGER_MCP_LEFT.x','RING_FINGER_MCP_LEFT.y','RING_FINGER_MCP_LEFT.z',
         'RING_FINGER_PIP_LEFT.x','RING_FINGER_PIP_LEFT.y','RING_FINGER_PIP_LEFT.z',
         'RING_FINGER_DIP_LEFT.x','RING_FINGER_DIP_LEFT.y','RING_FINGER_DIP_LEFT.z',
         'RING_FINGER_TIP_LEFT.x','RING_FINGER_TIP_LEFT.y','RING_FINGER_TIP_LEFT.z',
         'PINKY_MCP_LEFT.x','PINKY_MCP_LEFT.y','PINKY_MCP_LEFT.z',
         'PINKY_PIP_LEFT.x','PINKY_PIP_LEFT.y','PINKY_PIP_LEFT.z',
         'PINKY_DIP_LEFT.x','PINKY_DIP_LEFT.y','PINKY_DIP_LEFT.z',
         'PINKY_TIP_LEFT.x','PINKY_TIP_LEFT.y','PINKY_TIP_LEFT.z',
         'LEFT_score','RIGHT_score',
         'WRIST_RIGHT.x','WRIST_RIGHT.y','WRIST_RIGHT.z',
         'THUMB_CMC_RIGHT.x','THUMB_CMC_RIGHT.y','THUMB_CMC_RIGHT.z',
         'THUMB_MCP_RIGHT.x','THUMB_MCP_RIGHT.y','THUMB_MCP_RIGHT.z',
         'THUMB_IP_RIGHT.x','THUMB_IP_RIGHT.y','THUMB_IP_RIGHT.z',
         'THUMB_TIP_RIGHT.x','THUMB_TIP_RIGHT.y','THUMB_TIP_RIGHT.z',
         'INDEX_FINGER_MCP_RIGHT.x','INDEX_FINGER_MCP_RIGHT.y','INDEX_FINGER_MCP_RIGHT.z',
         'INDEX_FINGER_PIP_RIGHT.x','INDEX_FINGER_PIP_RIGHT.y','INDEX_FINGER_PIP_RIGHT.z',
         'INDEX_FINGER_DIP_RIGHT.x','INDEX_FINGER_DIP_RIGHT.y','INDEX_FINGER_DIP_RIGHT.z',
         'INDEX_FINGER_TIP_RIGHT.x','INDEX_FINGER_TIP_RIGHT.y','INDEX_FINGER_TIP_RIGHT.z',
         'MIDDLE_FINGER_MCP_RIGHT.x','MIDDLE_FINGER_MCP_RIGHT.y','MIDDLE_FINGER_MCP_RIGHT.z',
         'MIDDLE_FINGER_PIP_RIGHT.x','MIDDLE_FINGER_PIP_RIGHT.y','MIDDLE_FINGER_PIP_RIGHT.z',
         'MIDDLE_FINGER_DIP_RIGHT.x','MIDDLE_FINGER_DIP_RIGHT.y','MIDDLE_FINGER_DIP_RIGHT.z',
         'MIDDLE_FINGER_TIP_RIGHT.x','MIDDLE_FINGER_TIP_RIGHT.y','MIDDLE_FINGER_TIP_RIGHT.z',
         'RING_FINGER_MCP_RIGHT.x','RING_FINGER_MCP_RIGHT.y','RING_FINGER_MCP_RIGHT.z',
         'RING_FINGER_PIP_RIGHT.x','RING_FINGER_PIP_RIGHT.y','RING_FINGER_PIP_RIGHT.z',
         'RING_FINGER_DIP_RIGHT.x','RING_FINGER_DIP_RIGHT.y','RING_FINGER_DIP_RIGHT.z',
         'RING_FINGER_TIP_RIGHT.x','RING_FINGER_TIP_RIGHT.y','RING_FINGER_TIP_RIGHT.z',
         'PINKY_MCP_RIGHT.x','PINKY_MCP_RIGHT.y','PINKY_MCP_RIGHT.z',
         'PINKY_PIP_RIGHT.x','PINKY_PIP_RIGHT.y','PINKY_PIP_RIGHT.z',
         'PINKY_DIP_RIGHT.x','PINKY_DIP_RIGHT.y','PINKY_DIP_RIGHT.z',
         'PINKY_TIP_RIGHT.x','PINKY_TIP_RIGHT.y','PINKY_TIP_RIGHT.z']

In [ ]:
def save_in_csv(filename, list_results):
    header = results_header(list_results)
    with open(filename, 'w', encoding='UTF8', newline='') as f:
        writer = csv.DictWriter(f, fieldnames=header, delimiter=',')
        writer.writeheader()
        writer.writerows(list_results.values())
        
# function that union all the keys of the dictionaries in a set
def results_header(list_results):
    for timestamp in list_results:
        for key in list_results[timestamp]:
            if key not in header:
                header.append(key)
    return header

In [ ]:
#when two hands of eachother are detected
def Distance(x1,y1,x2,y2):
    Distance = np.linalg.norm(np.array([x1,y1])-np.array([x2,y2]))
    return Distance

def two_hands_each_other(hands_result, results):
    handness = results.multi_handedness
    if 'WRIST_LEFT.x' not in hands_result or 'WRIST_RIGHT.x' not in hands_result:
        return hands_result
    Distance_of_hands = Distance(hands_result['WRIST_LEFT.x'],hands_result['WRIST_LEFT.y'],hands_result['WRIST_RIGHT.x'],hands_result['WRIST_RIGHT.y'])
    if Distance_of_hands < 0.05 and len(handness)>1:
        if handness[0].classification[0].score > handness[1].classification[0].score:
            #toie aks chap o rast baraxe vase haminam inja bozorgaro pak mikonim 
            #because of fliping the image we have to remove the bigger one
            have_to_remove = handness[0].classification[0].label.upper()
        else:
            have_to_remove = handness[1].classification[0].label.upper()
        for key in list(hands_result.keys()):
            if 'score' in key:
                continue
            if have_to_remove in key:
                del hands_result[key]
    return hands_result

In [ ]:
#Detect left and right hand
#index: the hand result 0 or  1 hand: tedade ye dast ya 2ta
#the actual handlandmark results: all detections from model
def get_label(hand_index, results):
    handness = results.multi_handedness
    score = handness[hand_index].classification[0].score
    if len(handness)>1 and handness[0].classification[0].label == handness[1].classification[0].label:
        #results.multi_handedness[0].classification[0].label == right
        #results.multi_handedness[1].classification[0].label == left
        if handness[0].classification[0].score > handness[1].classification[0].score:
            if handness[1].classification[0].label == 'Left':
                handness[1].classification[0].label = 'Right'
            else:
                handness[1].classification[0].label = 'Left'
        else:
            if handness[0].classification[0].label == 'Left':
                handness[0].classification[0].label = 'Right'
            else:
                handness[0].classification[0].label = 'Left'
    #because i don't use flit in code 
    if handness[hand_index].classification[0].label == 'Right':
        return 'Left'.upper() 
    else:
        return 'Right'.upper() 

In [ ]:
def get_score (hand_index, results):
    handness = results.multi_handedness
    score = handness[hand_index].classification[0].score
    return score

In [ ]:
def fill_undetected_frames(list_results):
    # convert list_results to dataframe
    df = pd.DataFrame(list_results).T
    # fill empty values with NaN
    df = df.replace('', np.nan)
    # fill missing values (NaN) with previous value
    df = df.fillna(method='ffill').fillna(method='bfill')
    return df.T.to_dict()

fill_undetected_frames(list_results)

In [ ]:
# in this outputfile of mediapipe we put value of last frame when we don't have one of the hands
list_results = {}
total_nr_undetected_hands = 0
total_nr_image_with_undetected_hands = 0
def save_results(timestamp, results):
    #global ro mizanim k begim boro az birone function bareshdar
    nr_undetected_hand = 2 - len(results.multi_handedness)
    if nr_undetected_hand > 0:
        print('undetected hand: ', nr_undetected_hand)
        global total_nr_undetected_hands
        total_nr_undetected_hands = total_nr_undetected_hands + nr_undetected_hand
        
        global total_nr_image_with_undetected_hands
        total_nr_image_with_undetected_hands = total_nr_image_with_undetected_hands + 1
    for inx, hand in enumerate(results.multi_hand_landmarks): 
        which_hand = get_label(inx, results)
        score_of_hand = get_score(inx, results)
        hand_result = make_results_dict(timestamp, which_hand, hand)
        if timestamp not in list_results:
            list_results[timestamp] = {}
        hand_result[which_hand+'_score'] = score_of_hand
        list_results[timestamp].update(hand_result)
    list_results[timestamp] = two_hands_each_other(list_results[timestamp], results)

for path in images:
    with mp_hands.Hands(
    max_num_hands=2,
    model_complexity=1,
    #threshold for the initial detection to be successful
    min_detection_confidence=0.007, #0.001
    #threshold for the initial detection to be successful
    min_tracking_confidence=0.007) as hands: #0.001
        pic = cv2.imread(path, cv2.IMREAD_ANYCOLOR)
        image = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)  
        image.flags.writeable = False
        #image = cv2.flip(image,1)
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        file_name = os.path.basename(path)
        timestamp = file_name.split('.jpg')[0]
        timestamp = timestamp.split('shuttleFront_Kinect_ColorImage')[1]
        print(timestamp)
        if results.multi_hand_landmarks:
            save_results(timestamp, results)
        #cv2.imwrite(r'{}\detect{}'.format(output_directory, file_name), image)
save_in_csv(mediapipe_csv, list_results)
filled_list_results = fill_undetected_frames(list_results)
save_in_csv(mediapipe_csv.replace('.csv','_fill.csv'), filled_list_results)

In [ ]:
#merge mediapipe and 2dkinect csv
#filled_mediapipe_csv = pd.read_csv(mediapipe_csv.replace('.csv','_fill.csv'))
filled_mediapipe_csv = pd.read_csv(filled_mediapipe_csv)
kinect2D_csv = pd.read_csv(kinect_2D)
kinect2D_csv = kinect2D_csv.dropna(axis = 1)
merged = filled_mediapipe_csv.merge(kinect2D_csv, on='timestamp', how='outer')
merged.to_csv(merged_mediapipe_kinect, index=False)


In [ ]:
list_results

In [ ]:
#for visualistion
for path in images:
    with mp_hands.Hands(
    max_num_hands=2,
    model_complexity=1,
    #threshold for the initial detection to be successful
    min_detection_confidence=0.009,#0.001, 
    min_tracking_confidence=0.009) as hands: #0.001
        #while True:
        #for mediapipe we need RGB
        pic = cv2.imread(path, cv2.IMREAD_ANYCOLOR)
        image = cv2.cvtColor(pic,cv2.COLOR_BGR2RGB)  
        #flip on horizontal
        #image = cv2.flip(image,1)
        #set flag 
        image.flags.writeable = False
        #Detection
        results = hands.process(image)
        image.flags.writeable = True
        image = cv2.cvtColor(image,cv2.COLOR_RGB2BGR)
        print(results)
        if results.multi_hand_landmarks:
            for num, hand in enumerate(results.multi_hand_landmarks):
                for hand_landmarks in results.multi_hand_landmarks: 
                    mp_drawing.draw_landmarks(
                    image,
                    hand,
                    mp_hands.HAND_CONNECTIONS,
                    mp_drawing.DrawingSpec(color = (121,20,80),thickness=2, circle_radius=4),
                    mp_drawing.DrawingSpec(color = (122,22,250),thickness=2, circle_radius=4))
        file_name = os.path.basename(path)
        timestamp = file_name.split('.jpg')[0]
        output_images.append(file_name)
        # cv2.imshow(path, image)
        cv2.imwrite(r'{}\ {}'.format(output_directory, file_name),image)
        #print(timestamp, results.multi_handedness)
        # cv2.waitKey(0)
        #cv2.destroyAllWindows()

In [ ]:
# Kinect Joint hierarchy to mediapipe hand landmark dictionary
mapDict = {
    'WRIST': ['WRIST'],
    'THUMB': ['THUMB_CMC','THUMB_MCP','THUMB_IP','THUMB_TIP'],
    'HAND': ['INDEX_FINGER_MCP','INDEX_FINGER_MCP','RING_FINGER_MCP','PINKY_MCP'],
    'HANDTIP': ['INDEX_FINGER_TIP','MIDDLE_FINGER_TIP','RING_FINGER_TIP','PINKY_TIP'],

}


In [ ]:
def Edistance(x1,y1,x2,y2):
    x1 = float(x1)
    y1 = float(y1)
    x2 = float(x2)
    y2 = float(y2)
    Edistance = np.linalg.norm(np.array([x1,y1])-np.array([x2,y2]))
    return Edistance

list_of_results = []               
with open(mediapipe_csv) as f1, open(kinect_2D) as f2:
    rows_mediapipe = [{k: v for k, v in row1.items()} for row1 in csv.DictReader(f1, skipinitialspace=True)]
    rows_kinect = [{k: v for k, v in row2.items()} for row2 in csv.DictReader(f2, skipinitialspace=True)]
    #for row_mediapipe, row_kinect in zip(rows_mediapipe, rows_kinect):
    for row_mediapipe in rows_mediapipe:
        for row_kinect in rows_kinect:
            if row_mediapipe['timestamp'] == row_kinect['timestamp']:
                mDict = {}
                mDict['timestamp'] = row_mediapipe['timestamp']
                for kinect_id in mapDict:
                    for mediapipe_id in mapDict[kinect_id]:
                        for side in ['_LEFT','_RIGHT']:
                            value_xm = row_mediapipe[mediapipe_id+side+'.x']
                            if value_xm == '':
                                continue
                            value_xk = row_kinect[kinect_id+side+'.x']
                            value_ym = row_mediapipe[mediapipe_id+side+'.y']
                            value_yk = row_kinect[kinect_id+side+'.y']
                            mDict['d'+kinect_id+'_'+mediapipe_id+side] = Edistance(value_xm, value_ym, value_xk, value_yk)
                list_of_results.append(mDict)
#print(c)
#write in csv file
with open(compare_mediapipe_kinect, 'w', encoding='UTF8', newline='') as f3:
    writer = csv.DictWriter(f3,mDict.keys())
    writer.writeheader()
    writer.writerows(list_of_results)



In [ ]:
my_csv = pd.read_csv(compare_mediapipe_kinect)
mean_list= {}
for i in my_csv:
    if i == 'timestamp':
        continue
    else:
        column = my_csv[i]
        mean = column.mean()
        mean_list[i]=(mean)
        print(mean)
        

     
bins = np.linspace(-5, 5, 25, endpoint=True)
plt.bar(*zip(*mean_list.items()))
plt.rcParams.update({'font.size': 30})
plt.rcParams['figure.figsize'] = [50,50]
plt.xticks(rotation=90)
plt.xlabel(f'{total_nr_undetected_hands}undected hands, {total_nr_image_with_undetected_hands} image with detected hands')
plt.ylabel('mean of Euclidean distance')
plt.tight_layout()
plt.savefig('hist.png',dpi=200)
plt.show()



